# Redes Convolucionales

In [1]:
# Este notebook ha sido adptado del creado por: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
# Trabajamos de nuevo con MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf

In [4]:
# Los parámetros con los que trabajamos se asemejan al ejemplo de ayer de autoencoders
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

In [5]:
# Parámetros de la red, en este caso con una sola capa
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [6]:
# Definimos los tipos de variable
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [7]:
# Creacamos el modelo las funciones que lo definen, definimos las capas de convolución, 
#de agregación y la configuración relacionada.

# Capa convolucional
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))
# Capa de agregación
def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# Configuración de la red
def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Capa convolucional con unos pesos
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Capa de Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout supresión de salidas de la capa anterior
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Repetimos el proceso anterior de nuevo.
    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)

    # Capa completamente conectada con la capa anterior, es la capa final antes de la predicción
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # De nuevo con la función Relu de activación
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Capa de predicción de la clase
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [8]:
# Reservamos espacio para las variables, vemos que en este caso tenemos una configuración 
# diferente de pesos, por las características propias de las capas convolucionales y de sampling.
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
# Construimos el modelo utilizando la función que define la interacción entre capas.
pred = conv_net(x, weights, biases, keep_prob)

In [10]:
# La funcion de coste es la estimación de la perdida de entropia calculada en cada predicción.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [11]:
# Evaluamos el modelo comparando el valor real con la solución
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
# Inicializamos las variables, una vez hemos construido el grafo
init = tf.initialize_all_variables()

In [13]:
# Corremos el grafo
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Estapa de entrenamiento en función del limite iteracional
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Realizamos un proceso de segmentación de datos para realizar el entrenamiento por batchs.
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculamos el ajuste por cada muestra utilizada
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculamos la perdida al realizar la predicción sobre cada muestra
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.})

Iter 2560, Minibatch Loss= 22247.429688, Training Accuracy= 0.27344
Iter 5120, Minibatch Loss= 9276.361328, Training Accuracy= 0.54688
Iter 7680, Minibatch Loss= 12762.419922, Training Accuracy= 0.59375
Iter 10240, Minibatch Loss= 5497.883789, Training Accuracy= 0.73438
Iter 12800, Minibatch Loss= 5642.581543, Training Accuracy= 0.73438
Iter 15360, Minibatch Loss= 2443.253662, Training Accuracy= 0.81250
Iter 17920, Minibatch Loss= 2811.618652, Training Accuracy= 0.77344
Iter 20480, Minibatch Loss= 1594.819580, Training Accuracy= 0.80469
Iter 23040, Minibatch Loss= 2199.979736, Training Accuracy= 0.89062
Iter 25600, Minibatch Loss= 2484.760742, Training Accuracy= 0.82031
Iter 28160, Minibatch Loss= 2884.517578, Training Accuracy= 0.82812
Iter 30720, Minibatch Loss= 1801.751587, Training Accuracy= 0.85938
Iter 33280, Minibatch Loss= 2044.893555, Training Accuracy= 0.86719
Iter 35840, Minibatch Loss= 535.360840, Training Accuracy= 0.93750
Iter 38400, Minibatch Loss= 107.665466, Training A